# Calculating DRIS index

In [392]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations

## Data Processing

### Dummy Data

In [402]:
data_diagnosed_dict = {
    'P': [12,11,10],
    'Mg': [2,2,3],
    'N': [1,2,1],
    'Ca': [3,4,4],
    'Mn': [4,5,4]
}

data_optimum_dict = {
    'p': [12,11,10],
    'mg': [2,1,3],
    'N': [2,1,1],
    'ca': [4,4,3],
    'mn': [5,4,4],
}

df_diagnosed = pd.DataFrame(data_diagnosed_dict)
df_optimum = pd.DataFrame(data_optimum_dict)


### Calculate ratios

In [403]:
def calculate_ratios(df):
    # Step 2: Calculate the ratios
    ratios = {}
    for col1, col2 in combinations(df.columns, 2):
        ratio_name1 = f"{col1}/{col2}"
        # ratio_name2 = f"{col2}/{col1}"
        ratios[ratio_name1] = df[col1] / df[col2]
        # ratios[ratio_name2] = df[col2] / df[col1]

    return pd.DataFrame(ratios)

df_optimum_ratios = calculate_ratios(df_optimum)
df_diagnosed_ratios = calculate_ratios(df_diagnosed)


### Calculate coefficient of variation of the optimum values

In [404]:
def calculate_CV(df_ratios):
    cv_dict = {}
    for ratio_name, ratio_values in df_ratios.items():
        mean_ratio = ratio_values.mean()
        std_ratio = ratio_values.std()
        cv = (std_ratio / mean_ratio) * 100  # percentage or not 
        cv_dict[ratio_name] = [cv]
    return pd.DataFrame(cv_dict)

df_optimum_CV = calculate_CV(df_optimum_ratios)
df_optimum_CV.head()

,p/mg,p/N,p/ca,p/mn,mg/N,mg/ca,mg/mn,N/ca,N/mn,ca/mn
0,57.423868,29.397237,9.665737,7.069708,69.282032,65.465367,54.981444,35.250582,28.867513,15.563243


### Calculate rations of the mean

In [405]:
df_op = pd.DataFrame(df_optimum.mean()).T
df_optimum_mean_ratios = calculate_ratios(df_op)

df_di = pd.DataFrame(df_diagnosed.mean()).T
df_diagnosed_mean_ratios = calculate_ratios(df_di)


## Calculate f(A/B) 

\begin{align}
    f(A/B) = 
    \begin{cases}
    &\biggl(\frac{A/B}{a/b} - 1\biggr)\frac{1000}{CV} \quad A/B \ge a/b\\
    &\biggl(1 - \frac{a/b}{A/B}\biggr)\frac{1000}{CV} \quad A/B < a/b
    \end{cases}
\end{align}

- $A/B$ is the ratio of two elements (A and B) in the tissue of the pant being diagnosed.
- $a/b$ is the ration of the optimal values 
- CV is the coefficient of variation of the optimal values $a$ and $b$: $CV = \sigma/\mu$ 

In [406]:
def f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV):
    names = df_diagnosed_mean_ratios.columns
    diagnosed_ratios = df_diagnosed_mean_ratios.to_numpy()
    optimum_mean_ratios = df_optimum_mean_ratios.to_numpy()
    optimum_CV = df_optimum_CV.to_numpy()

    def f_single(diagnosed_ratio, optimum_mean_ratio, CV):
        if diagnosed_ratio == 0:
            return float('inf')  # Return inf or some large number to handle zero division
        if diagnosed_ratio >= optimum_mean_ratio:
            f = ((diagnosed_ratio / optimum_mean_ratio) - 1) * 1000 / CV
        else:
            f = (1 - (optimum_mean_ratio / diagnosed_ratio)) * 1000 / CV
        return [f]

    f_dict = dict()
    for i in range(len(names)):
        f_dict[f"f({names[i]})"] = f_single(diagnosed_ratios[0, i], optimum_mean_ratios[0, i], optimum_CV[0, i])
    return pd.DataFrame(f_dict)


# Example usage
# Assuming df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV are defined DataFrames with similar structure
df_f = f(df_diagnosed_mean_ratios, df_optimum_mean_ratios, df_optimum_CV)
df_f.head()


,f(P/Mg),f(P/N),f(P/Ca),f(P/Mn),f(Mg/N),f(Mg/Ca),f(Mg/Mn),f(N/Ca),f(N/Mn),f(Ca/Mn)
0,-2.902394,0.0,0.0,0.0,2.405626,2.545875,3.031326,0.0,0.0,0.0


## Calculate DRIS index


$\mathrm{A_{index}} = \frac{1}{z} \bigl[f(A/B) + f(A/C) + f(A/D) + \cdots + f(A/N)\bigr]$

In [407]:
def create_index_string(index_element, df_diagnosed):
    elements = df_diagnosed.columns
    result_string = f'I_{elements[index_element]} ='
    for i, element in enumerate(elements):
        if index_element < i:
            result_string += (f' + f({elements[index_element]}/{element})')
        elif index_element > i:
            result_string += (f' - f({element}/{elements[index_element]})')
    return result_string

result_string = create_index_string(0,df_diagnosed)
result_string

'I_P = + f(P/Mg) + f(P/N) + f(P/Ca) + f(P/Mn)'

In [408]:
def calculate_index_value(index_element, df_diagnosed, df_f):
    f_dict = df_f.to_dict('index')[0]
    elements = df_diagnosed.columns
    result = 0
    for i, element in enumerate(elements):
        if index_element < i:
            result += f_dict[f'f({elements[index_element]}/{element})']
        elif index_element > i:
            result -= f_dict[f'f({element}/{elements[index_element]})']
    return result

In [409]:
def calculate_all_index_values(df_diagnosed, df_f):
    elements = df_diagnosed.columns
    results_dict = dict()
    for i in range(len(elements)):
        results_dict[f'I_{elements[i]}'] = [calculate_index_value(i, df_diagnosed, df_f)]
    return pd.DataFrame(results_dict)

DRIS_indices = calculate_all_index_values(df_diagnosed, df_f)

DRIS_indices.head()

,I_P,I_Mg,I_N,I_Ca,I_Mn
0,-2.902394,10.885221,-2.405626,-2.545875,-3.031326


## Export DRIS indices

In [401]:
# DRIS_indices.to_csv('dris.csv', index=False)
DRIS_indices.to_excel('dris.xlsx', index=False)